# Grid search CV

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Loading data

In [23]:
df = sns.load_dataset("iris")
df.head()
#We will be predicting species

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# train test split and dropping output values

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X = df.drop("species", axis=1)
y = df["species"]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# KNN 

In [27]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors = 5)

model_knn.fit(X_train, y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [28]:
model_knn.score(X_test, y_test)

1.0

# SVM

In [29]:
from sklearn.svm import SVC
model_svm = SVC(gamma = "auto")

model_svm.fit(X_train, y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'auto'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [30]:
model_svm.score(X_test, y_test)

1.0

In [31]:
#Now lets use grid search CV
from sklearn.model_selection import GridSearchCV
classifier = GridSearchCV((model_svm), {  # these all values are being added using the docs
    "C": [1,10,20,30],
    "kernel": ["rbf", "linear"],
}, cv = 5, return_train_score = False)

In [32]:
classifier.fit(X, y)

,estimator,SVC(gamma='auto')
,param_grid,"{'C': [1, 10, ...], 'kernel': ['rbf', 'linear']}"
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,1


In [33]:
classifier.cv_results_

{'mean_fit_time': array([0.00289545, 0.00225573, 0.00185456, 0.00255218, 0.00255265,
        0.0015799 , 0.00135946, 0.00139785]),
 'std_fit_time': array([0.00030274, 0.00025617, 0.00097957, 0.0004861 , 0.00019383,
        0.00062742, 0.00026889, 0.00025266]),
 'mean_score_time': array([0.00207682, 0.00182505, 0.00112963, 0.00197635, 0.00169415,
        0.00105567, 0.00107284, 0.00100813]),
 'std_score_time': array([0.00038262, 0.00055458, 0.00038696, 0.00036162, 0.00043833,
        0.00037372, 0.00021124, 0.00010253]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20, 30, 30],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear',
                    'rbf', 'linear'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C

In [34]:
#Here we have converted the result into the df so that we can see it properly
result = pd.DataFrame(classifier.cv_results_)
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002895,0.000303,0.002077,0.000383,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.002256,0.000256,0.001825,0.000555,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001855,0.000980,0.001130,0.000387,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.002552,0.000486,0.001976,0.000362,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.002553,0.000194,0.001694,0.000438,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.001580,0.000627,0.001056,0.000374,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6
6,0.001359,0.000269,0.001073,0.000211,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.933333,1.0,0.960000,0.038873,7
7,0.001398,0.000253,0.001008,0.000103,30,linear,"{'C': 30, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.900000,1.0,0.960000,0.048990,7


In [35]:
#Here we have printed only those col which we needed
result[["param_C", "param_kernel", "mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667
6,30,rbf,0.960000
7,30,linear,0.960000


# Random search CV

In [36]:
#In random search CV, we will get random results of how many we have specified
from sklearn.model_selection import RandomizedSearchCV
classifier_r = RandomizedSearchCV((model_svm),{
    "C": [1,10,20,30],
    "kernel": ["rbf", "linear"],
}, n_iter=4, cv=5, return_train_score = False)

In [37]:
classifier_r.fit(X,y)

,estimator,SVC(gamma='auto')
,param_distributions,"{'C': [1, 10, ...], 'kernel': ['rbf', 'linear']}"
,n_iter,4
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [38]:
classifier_r.cv_results_

{'mean_fit_time': array([0.0033308 , 0.0027854 , 0.00232658, 0.00204625]),
 'std_fit_time': array([0.00108543, 0.00035497, 0.00120191, 0.00063048]),
 'mean_score_time': array([0.00260077, 0.00176864, 0.00188861, 0.00125728]),
 'std_score_time': array([0.00095189, 0.0005252 , 0.00088856, 0.00021303]),
 'param_kernel': masked_array(data=['rbf', 'linear', 'linear', 'linear'],
              mask=[False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'param_C': masked_array(data=[1, 1, 20, 30],
              mask=[False, False, False, False],
        fill_value=999999),
 'params': [{'kernel': 'rbf', 'C': 1},
  {'kernel': 'linear', 'C': 1},
  {'kernel': 'linear', 'C': 20},
  {'kernel': 'linear', 'C': 30}],
 'split0_test_score': array([0.96666667, 0.96666667, 1.        , 1.        ]),
 'split1_test_score': array([1., 1., 1., 1.]),
 'split2_test_score': array([0.96666667, 0.96666667, 0.9       , 0.9       ]),
 'split3_test_score': array([0.96666667, 0.96666

In [39]:
results = pd.DataFrame(classifier_r.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003331,0.001085,0.002601,0.000952,rbf,1,"{'kernel': 'rbf', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.002785,0.000355,0.001769,0.000525,linear,1,"{'kernel': 'linear', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.002327,0.001202,0.001889,0.000889,linear,20,"{'kernel': 'linear', 'C': 20}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,3
3,0.002046,0.000630,0.001257,0.000213,linear,30,"{'kernel': 'linear', 'C': 30}",1.000000,1.0,0.900000,0.900000,1.0,0.960000,0.048990,4


In [40]:
results[["param_C", "param_kernel", "mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,20,linear,0.966667
3,30,linear,0.960000


# Ensemble Learning 

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns


In [44]:
base_learners = [
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('svc', SVC(probability=True, kernel='rbf', random_state=42)),
    ('lr', LogisticRegression(max_iter=1000))
]

In [47]:
meta_learner = LogisticRegression(max_iter=1000)

# Stacking Classifier

In [48]:
stacking_clf = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    cv=5
)


In [51]:
stacking_clf.fit(X_train, y_train)

,estimators,"[('dt', ...), ('svc', ...), ...]"
,final_estimator,LogisticRegre...max_iter=1000)
,cv,5
,stack_method,'auto'
,n_jobs,None
,passthrough,False
,verbose,0
,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2


In [53]:
y_pred = stacking_clf.predict(X_test)

In [54]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

1.0

# Random forest classifier

In [55]:
from sklearn.ensemble import RandomForestClassifier

In [59]:
model_rf = RandomForestClassifier(
    n_estimators=100,        # number of trees
    max_depth=None,          # let trees grow fully
    random_state=42
)
model_rf.fit(X_train, y_train)


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [61]:
y_pred = model_rf.predict(X_test)

In [62]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

1.0